In [ ]:
import mysql.connector
import hashlib
import getpass

users = {
    "admin": {"password": hashlib.md5("adminpassword".encode()).hexdigest(), "role": "admin"},
}
cnct = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Root@123",
    database="hospitalmanagement",
    buffered=True  # This option enables automatic fetching of results
)

cursor = cnct.cursor(dictionary=True)  
# Use dictionary cursor for easier result access
# Function to get existing IDs for validation
def get_existing_ids(table_name, id_column):
    cursor.execute(f"SELECT {id_column} FROM {table_name}")
    return [str(row[id_column]) for row in cursor.fetchall()]


create_patient_tables = '''
CREATE TABLE IF NOT EXISTS patients (
    patient_id INT PRIMARY KEY,
    name VARCHAR(20),
    age BIGINT,
    address VARCHAR(50),
    contact BIGINT, 
    height FLOAT,
    weight FLOAT,
    medical_history VARCHAR(255),
    email VARCHAR(20)  
);
'''

cursor.execute(create_patient_tables)
cnct.commit()

create_doctor_tables = '''
CREATE TABLE IF NOT EXISTS doctors (
    doc_id INT PRIMARY KEY,
    name VARCHAR(20),
    specialization VARCHAR(20),
    address VARCHAR(50),
    contact BIGINT,
    availability VARCHAR(255),
    salary INT(11),
    email VARCHAR(50),  
    years_of_experience INT  
);
'''
cursor.execute(create_doctor_tables)
cnct.commit()

create_staff_tables = '''
CREATE TABLE IF NOT EXISTS staff (
    staff_id INT PRIMARY KEY,
    name VARCHAR(50),
    designation VARCHAR(20),
    address VARCHAR(50),
    contact BIGINT,
    department VARCHAR(50),
    salary INT(11),
    email VARCHAR(50)
);
'''

cursor.execute(create_staff_tables)
cnct.commit()

create_nurse_tables = '''
CREATE TABLE IF NOT EXISTS nurses (
    nurse_id INT PRIMARY KEY,
    name VARCHAR(50),
    address VARCHAR(50),
    contact BIGINT,
    department VARCHAR(25),
    salary INT(11),
    doctor_id INT,
    FOREIGN KEY (doctor_id) REFERENCES doctors (doc_id) ,
    email VARCHAR(50)
);
'''

cursor.execute(create_nurse_tables)
cnct.commit()

create_appointment_tables = '''
CREATE TABLE IF NOT EXISTS appointments (
    app_id INT PRIMARY KEY,
    medical_issue VARCHAR(255),
    doctor_id INT,
    patient_id INT,
    date DATE,
    time TIME,
    FOREIGN KEY (doctor_id) REFERENCES doctors (doc_id),
    FOREIGN KEY (patient_id) REFERENCES patients (patient_id) 
);
'''

cursor.execute(create_appointment_tables)
cnct.commit()

create_billing_tables = '''
CREATE TABLE IF NOT EXISTS billing (
    bill_id INT PRIMARY KEY,
    appointment_id INT,
    patient_id INT,
    total_amount FLOAT,
    payment_status VARCHAR(75),
    FOREIGN KEY (appointment_id) REFERENCES appointments (app_id),
    FOREIGN KEY (patient_id) REFERENCES patients (patient_id) 
);
'''

cursor.execute(create_billing_tables)
cnct.commit()

create_user_table = '''
CREATE TABLE IF NOT EXISTS users (
    user_id INT AUTO_INCREMENT PRIMARY KEY,
    username VARCHAR(255) NOT NULL,
    password VARCHAR(255) NOT NULL
);
'''

cursor.execute(create_user_table)
cnct.commit()



# Function for Adding a doctor
def add_doctor():
    

    try:
        doc_id = input("Enter doctor ID: ")
        if doc_id=='':
            print("Enter Valid ID :")
            doc_id = input("Enter doctor ID: ")
        # Check if the doctor ID already exists
        cursor.execute("SELECT COUNT(*) FROM doctors WHERE doc_id = %s", (doc_id,))
        result_count = cursor.fetchone()['COUNT(*)']

        if result_count > 0:
            print("Doctor with this ID already exists. Please choose a different ID.")
            return


        name = input("Enter doctor name: ")
        specialization = input("Enter doctor specialization: ")
        address = input("Enter doctor address: ")
        contact = input("Enter doctor contact: ")

        # Validate contact length
        if len(contact) > 10:
            print("Please enter a valid contact number.")
            contact = input("Enter doctor contact: ")


        availability = input("Enter doctor availability: ")
        salary = int(input("Enter doctor monthly salary: "))
        email = input("Enter doctor email: ")
        years_of_experience = int(input("Enter doctor years of experience: "))


        # Insert data into the doctors table
        query = "INSERT INTO doctors (doc_id, name, specialization, address, contact, availability, salary, email, years_of_experience) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
        values = (doc_id, name, specialization, address, contact, availability, salary, email, years_of_experience)

        cursor.execute(query, values)
        cnct.commit()
        print("Doctor added successfully.")
        
    except mysql.connector.Error as err:
        print("Failed to excecute request please try again .")
    except ValueError:
        print("Invalid input. Please enter a valid value.")


def add_nurse():
    try:
        nurse_id = input("Enter nurse ID: ")

        # Check if the nurse ID already exists
        cursor.execute("SELECT COUNT(*) FROM nurses WHERE nurse_id = %s", (nurse_id,))
        result_count = cursor.fetchone()['COUNT(*)']

        if result_count > 0:
            print("Nurse with this ID already exists. Please choose a different ID.")
            return

        name = input("Enter nurse name: ")
        address = input("Enter nurse address: ")
        contact = input("Enter nurse contact: ")

        # Validate contact length
        if len(contact) > 10:
            print("Please enter a valid contact number.")
            contact = input("Enter nurse contact: ")


        department = input("Enter nurse department: ")
        salary = int(input("Enter nurse salary: "))
        doctor_id = int(input("Enter doctor ID (if applicable, otherwise enter 0): "))
        email = input("Enter nurse email: ")

        # Insert data into the nurses table
        query = "INSERT INTO nurses (name,nurse_id, address, contact, department, salary, doctor_id, email) VALUES (%s,%s, %s, %s, %s, %s, %s, %s)"
        values = (name,nurse_id,address, contact, department, salary, doctor_id, email)

        cursor.execute(query, values)
        cnct.commit()


        print("Nurse added successfully.")
    except mysql.connector.Error as err:
        print("Failed to excecute request please try again .")
    except ValueError:
        print("Invalid input. Please enter a valid value.")

def add_staff():
    try:
        staff_id = input("Enter staff ID: ")

        # Check if the staff ID already exists
        cursor.execute("SELECT COUNT(*) FROM staff WHERE staff_id = %s", (staff_id,))
        result_count = cursor.fetchone()['COUNT(*)']

        if result_count > 0:
            print(" with this ID already exists. Please choose a different ID.")
            return
        name = input("Enter staff name: ")
        contact = input("Enter staff contact: ")

        # Validate contact length
        if len(contact) > 10:
            print("Please enter a valid contact number.")
            contact = input("Enter staff contact: ")

        email = input("Enter staff email: ")
        department = input("Enter staff department: ")
        designation = input("Enter staff designation: ")
        address = input("Enter staff address: ")
        salary = int(input("Enter staff salary: "))

        # Insert data into the staff table
        query = "INSERT INTO staff (name,staff_id, contact, email, department, designation, address, salary) VALUES (%s,%s, %s, %s, %s, %s, %s, %s)"
        values = (name,staff_id, contact, email, department, designation, address, salary)

        cursor.execute(query, values)
        cnct.commit()

        print("Staff added successfully.")
    except mysql.connector.Error as err:
        print("Failed to excecute request please try again .")
    except ValueError:
        print("Invalid input. Please enter a valid value.")
        
# Function for Adding a patient
def add_patient():
    try:
        patient_id=input("Enter parient ID:")
        name = input("Enter patient name: ")
        age = int(input("Enter patient age: "))
        address = input("Enter patient address: ")
        contact = input("Enter patient contact: ")

        # Validate contact length
        if len(contact) > 10:
            print("Please enter a valid contact number.")
            return

        height = float(input("Enter patient height (in cm): "))
        weight = float(input("Enter patient weight (in kg): "))
        medical_history = input("Enter patient medical history: ")
        email = input("Enter patient email: ")

        # Insert data into the patients table
        query = "INSERT INTO patients (name,patient_id, age, address, contact, height, weight, medical_history, email) VALUES (%s,%s, %s, %s, %s, %s, %s, %s, %s)"
        values = (name,patient_id, age, address, contact, height, weight, medical_history, email)

        cursor.execute(query, values)
        cnct.commit()

        print("Patient added successfully.")
    except mysql.connector.Error as err:
        print("Failed to execute request. Please try again.")
    except ValueError:
        print("Invalid input. Please enter a valid value.")



# Function for adding an appointment
def book_appointment():
    try:
        app_id = input("Enter appointment ID: ")
        medical_issue = input("Enter medical issue: ")

        # Get existing doctor IDs for validation
        existing_doctor_ids = get_existing_ids("doctors", "doc_id")

        doctor_id = input("Enter doctor ID: ")

        # Validate doctor ID
        if doctor_id not in existing_doctor_ids:
            print("Doctor with this ID does not exist. Please enter a valid doctor ID.")
            return

        # Get existing patient IDs for validation
        existing_patient_ids = get_existing_ids("patients", "patient_id")

        patient_id = input("Enter patient ID: ")

        # Validate patient ID
        if patient_id not in existing_patient_ids:
            print("Patient with this ID does not exist. Please enter a valid patient ID.")
            return

        date = input("Enter appointment date (YYYY-MM-DD): ")
        time = input("Enter appointment time (HH:MM:SS): ")

        query = "INSERT INTO appointments (app_id, medical_issue, doctor_id, patient_id, date, time) VALUES (%s, %s, %s, %s, %s, %s)"
        values = (app_id, medical_issue, doctor_id, patient_id, date, time)

        cursor.execute(query, values)
        cnct.commit()
        print("Appointment booked successfully.")  
    except mysql.connector.Error as err:
        print("Failed to excecute request please try again .")
    except ValueError:
        print("Invalid input. Please enter a valid value.")

# Function for adding billing information
def add_billing():
    try:
        bill_id = input("Enter billing ID: ")

        # Get existing appointment IDs for validation
        existing_appointment_ids = get_existing_ids("appointments", "app_id")

        appointment_id = input("Enter appointment ID: ")

        # Validate appointment ID
        if appointment_id not in existing_appointment_ids:
            print("Appointment with this ID does not exist. Please enter a valid appointment ID.")
            return

        # Get existing patient IDs for validation
        existing_patient_ids = get_existing_ids("patients", "patient_id")

        patient_id = input("Enter patient ID: ")

        # Validate patient ID
        if patient_id not in existing_patient_ids:
            print("Patient with this ID does not exist. Please enter a valid patient ID.")
            return

        total_amount = float(input("Enter total amount: "))
        payment_status = input("Enter payment status: ")

        query = "INSERT INTO billing (bill_id, appointment_id, patient_id, total_amount, payment_status) VALUES (%s, %s, %s, %s, %s)"
        values = (bill_id, appointment_id, patient_id, total_amount, payment_status)

        cursor.execute(query, values)
        cnct.commit()

        print("Billing information added successfully.")
    except mysql.connector.Error as err:
        print("Failed to excecute request please try again .")
    except ValueError:
        print("Invalid input. Please enter a valid value.")


# Function for updating a doctor's details
def update_doctor():
    try:
        existing_doctor_ids = get_existing_ids("doctors", "doc_id")

        doc_id = input("Enter the ID of the doctor you want to update: ")
        if doc_id not in existing_doctor_ids:
            print("Doctor with this ID does not exist. Please enter a valid doctor ID.")
            return

        name = input("Enter updated name: ")
        specialization = input("Enter updated specialization: ")
        address = input("Enter updated address: ")
        contact = input("Enter updated contact: ")

        # Validate contact length
        if len(contact) > 10:
            print("Please enter a valid contact number.")
            return

        availability = input("Enter updated availability: ")
        salary = int(input("Enter updated salary: "))
        email = input("Enter updated email: ")
        years_of_experience = int(input("Enter updated years of experience: "))

        update_query = "UPDATE doctors SET name=%s, specialization=%s, address=%s, contact=%s, availability=%s, salary=%s, email=%s, years_of_experience=%s WHERE doc_id=%s"
        update_values = (name, specialization, address, contact, availability, salary, email, years_of_experience, doc_id)

        cursor.execute(update_query, update_values)
        cnct.commit()

        print("Doctor details updated successfully.")
    except mysql.connector.Error as err:
        print("Failed to execute request. Please try again.")
    except ValueError:
        print("Invalid input. Please enter a valid value.")
        
        
# Function for updating a staff member's details
def update_staff():
    try:
        staff_id = input("Enter the ID of the staff member you want to update: ")
        name = input("Enter updated name: ")
        designation = input("Enter updated designation: ")
        address = input("Enter updated address: ")
        contact = input("Enter updated contact: ")

        # Validate contact length
        if len(contact) > 10:
            print("Please enter a valid contact number.")
            return

        department = input("Enter updated department: ")
        position = input("Enter updated position: ")
        salary = int(input("Enter updated salary: "))
        email = input("Enter updated email: ")

        update_query = "UPDATE staff SET name=%s, designation=%s, address=%s, contact=%s, department=%s, position=%s, salary=%s, email=%s WHERE staff_id=%s"
        update_values = (name, designation, address, contact, department, position, salary, email, staff_id)

        cursor.execute(update_query, update_values)
        cnct.commit()

        print("Staff member details updated successfully.")
    except mysql.connector.Error as err:
        print("Failed to execute request. Please try again.")
    except ValueError:
        print("Invalid input. Please enter a valid value.")

# Function for updating a patient's details
def update_patient():
    try:
        existing_patient_ids = get_existing_ids("patients", "patient_id")
        patient_id = input("Enter the ID of the patient you want to update: ")
        if patient_id not in existing_patient_ids:
            print("Patient with this ID does not exist. Please enter a valid patient ID.")
            return

        name = input("Enter updated name: ")
        age = int(input("Enter updated age: "))
        address = input("Enter updated address: ")
        contact = input("Enter updated contact: ")

        # Validate contact length
        if len(contact) > 10:
            print("Please enter a valid contact number.")
            return

        height = float(input("Enter updated height (in cm): "))
        weight = float(input("Enter updated weight (in kg): "))
        medical_history = input("Enter updated medical history: ")
        email = input("Enter updated email: ")  # Added email input

        update_query = "UPDATE patients SET name=%s, age=%s, address=%s, contact=%s, height=%s, weight=%s, medical_history=%s, email=%s WHERE patient_id=%s"
        update_values = (name, age, address, contact, height, weight, medical_history, email, patient_id)

        cursor.execute(update_query, update_values)
        cnct.commit()

        print("Patient details updated successfully.")
    except mysql.connector.Error as err:
        print("Failed to execute request. Please try again.")
    except ValueError:
        print("Invalid input. Please enter a valid value.")

    

# Function for updating a nurse's details
def update_nurse():
    try:
        nurse_id = input("Enter the ID of the nurse you want to update: ")
        name = input("Enter updated name: ")
        address = input("Enter updated address: ")
        contact = input("Enter updated contact: ")

        # Validate contact length
        if len(contact) > 10:
            print("Please enter a valid contact number.")
            return

        department = input("Enter updated department: ")
        salary = int(input("Enter updated salary: "))
        doctor_id = int(input("Enter updated doctor ID (if applicable, otherwise enter 0): "))  # Assuming doctor_id is an integer
        email = input("Enter updated email: ")

        update_query = "UPDATE nurses SET name=%s, address=%s, contact=%s, department=%s, salary=%s, doctor_id=%s, email=%s WHERE nurse_id=%s"
        update_values = (name, address, contact, department, salary, doctor_id, email, nurse_id)

        cursor.execute(update_query, update_values)
        cnct.commit()

        print("Nurse details updated successfully.")
    except mysql.connector.Error as err:
        print("Failed to execute request. Please try again.")
    except ValueError:
        print("Invalid input. Please enter a valid value.")
        
def delete_doctor():
    try:
        doc_id = input("Enter the ID of the doctor you want to delete: ")
        existing_doctor_ids = get_existing_ids("doctors", "doc_id")
        
        if doc_id not in existing_doctor_ids:
            print("Doctor with this ID does not exist. Please enter a valid doctor ID.")
            return
        
        # Update the doctor_id in the nurses table to null
        update_nurses_query = "UPDATE nurses SET doctor_id = NULL WHERE doctor_id = %s"
        cursor.execute(update_nurses_query, (doc_id,))
        cnct.commit()
        
        # Get the related appointments
        get_appointments_query = "SELECT app_id FROM appointments WHERE doctor_id = %s"
        cursor.execute(get_appointments_query, (doc_id,))
        app_ids = [row['app_id'] for row in cursor.fetchall()]
        
        # Delete the related appointments
        if app_ids:
            delete_appointments_query = "DELETE FROM appointments WHERE app_id IN ({})".format(', '.join(['%s']*len(app_ids)))
            cursor.execute(delete_appointments_query, app_ids)
            cnct.commit()
        
        # Delete the doctor
        delete_doctor_query = "DELETE FROM doctors WHERE doc_id = %s"
        cursor.execute(delete_doctor_query, (doc_id,))
        cnct.commit()
        
        print("Doctor deleted successfully.")
    except mysql.connector.Error as err:
        print("Failed to execute request. Please try again.")
        
        
def print_table(table_name):
    try:
        query = f"SELECT * FROM {table_name}"
        cursor.execute(query)
        records = cursor.fetchall()
        if not records:
            print(f"No records available for {table_name}.")
            return
        columns = [desc[0] for desc in cursor.description]
        column_widths = [max(len(str(record[column])) for record in records) for column in columns]
        for i, column in enumerate(columns):
            print(f"{column.ljust(column_widths[i])}", end=' ')
        print("\n" + "-" * (sum(column_widths) + len(columns) - 1))
        for record in records:
            for i, column in enumerate(columns):
                print(f"{str(record[column]).ljust(column_widths[i])}", end=' ')
            print()
    except mysql.connector.Error as err:
        print(f"Failed to execute request for {table_name}. Please try again.")
        
        
def update_appointment():
    try:
        app_id = input("Enter appointment ID to update: ")

        # Check if the appointment ID exists
        existing_appointment_ids = get_existing_ids("appointments", "app_id")
        if app_id not in existing_appointment_ids:
            print("Appointment with this ID does not exist. Please enter a valid appointment ID.")
            return

        new_date = input("Enter new appointment date (YYYY-MM-DD): ")
        new_time = input("Enter new appointment time (HH:MM:SS): ")

        query = "UPDATE appointments SET date = %s, time = %s WHERE app_id = %s"
        values = (new_date, new_time, app_id)

        cursor.execute(query, values)
        cnct.commit()

        print("Appointment updated successfully.")
    except mysql.connector.Error as err:
        print("Failed to excecute request please try again .")
    except ValueError:
        print("Invalid input. Please enter a valid value.")


    
# Function for Updating billing information
def update_billing():
    try:
        bill_id = input("Enter billing ID to update: ")

        # Check if the billing ID exists
        existing_billing_ids = get_existing_ids("billing", "bill_id")
        if bill_id not in existing_billing_ids:
            print("Billing with this ID does not exist. Please enter a valid billing ID.")
            return

        new_amount = float(input("Enter new billing amount: "))
        new_payment_status = input("Enter new payment status: ")

        query = "UPDATE billing SET total_amount = %s, payment_status = %s WHERE bill_id = %s"
        values = (new_amount, new_payment_status, bill_id)

        cursor.execute(query, values)
        cnct.commit()

        print("Billing information updated successfully.")
    except mysql.connector.Error as err:
        print("Failed to excecute request please try again .")
    except ValueError:
        print("Invalid input. Please enter a valid value.")


# Function for deleting a patient
def delete_patient():
    patient_id = input("Enter the ID of the patient you want to delete: ")

    # Get the related appointments
    get_appointments_query = "SELECT app_id FROM appointments WHERE patient_id = %s"
    cursor.execute(get_appointments_query, (patient_id,))
    app_ids = [row['app_id'] for row in cursor.fetchall()]

    # Delete the related appointments
    if app_ids:
        delete_appointments_query = "DELETE FROM appointments WHERE app_id IN ({})".format(', '.join(['%s']*len(app_ids)))
        cursor.execute(delete_appointments_query, app_ids)
        cnct.commit()

    # Get the related billing
    get_billing_query = "SELECT bill_id FROM billing WHERE patient_id = %s"
    cursor.execute(get_billing_query, (patient_id,))
    bill_ids = [row['bill_id'] for row in cursor.fetchall()]

    # Delete the related billing
    if bill_ids:
        delete_billing_query = "DELETE FROM billing WHERE bill_id IN ({})".format(', '.join(['%s']*len(bill_ids)))
        cursor.execute(delete_billing_query, bill_ids)
        cnct.commit()

    # Delete the patient
    delete_query = "DELETE FROM patients WHERE patient_id = %s"
    delete_values = (patient_id,)
    cursor.execute(delete_query, delete_values)
    print("Patient and related appointments and billing are deleted successfully !!")
    cnct.commit()



def delete_nurse():
    try:
        nurse_id = input("Enter the ID of the nurse you want to delete: ")

        existing_nurse_ids = get_existing_ids("nurses", "nurse_id")
        if nurse_id not in existing_nurse_ids:
            print("Nurse with this ID does not exist. Please enter a valid Nurse ID.")
            return

        delete_query = "DELETE FROM nurses WHERE nurse_id = %s"
        delete_values = (nurse_id,)
        cursor.fetchall()
        cursor.execute(delete_query, delete_values)
        cnct.commit()

        print("Nurse deleted successfully.")
    except mysql.connector.Error as err:
        print("Failed to excecute request please try again .")
    except ValueError:
        print("Invalid input. Please enter a valid value.")

    
def delete_appointment():
    try:
        app_id = input("Enter appointment ID: ")

        # Check if the appointment ID exists
        existing_appointment_ids = get_existing_ids("appointments", "app_id")
        if app_id not in existing_appointment_ids:
            print("Appointment with this ID does not exist. Please enter a valid appointment ID.")
            return


        # Update billing, setting appointment_id to NULL instead of deleting
        update_billing_query = "UPDATE billing SET appointment_id = NULL WHERE appointment_id = %s"
        cursor.execute(update_billing_query, (app_id,))
        cnct.commit()

        # Delete the appointment
        delete_query = "DELETE FROM appointments WHERE app_id = %s"
        cursor.execute(delete_query, (app_id,))
        cnct.commit()

        print("Appointment deleted successfully. Related billing records were updated.")
    except mysql.connector.Error as err:
        print("Failed to excecute request please try again .")
    except ValueError:
        print("Invalid input. Please enter a valid value.")


def delete_billing():
    try:
        bill_id = input("Enter billing ID to delete: ")

        # Check if the billing ID exists
        existing_billing_ids = get_existing_ids("billing", "bill_id")
        if bill_id not in existing_billing_ids:
            print("Billing with this ID does not exist. Please enter a valid billing ID.")
            return

        query = "DELETE FROM billing WHERE bill_id = %s"
        values = (bill_id,)
        cursor.execute(query, values)
        cnct.commit()

        print("Billing information deleted successfully.")
    except mysql.connector.Error as err:
        print("Failed to excecute request please try again .")
    except ValueError:
        print("Invalid input. Please enter a valid value.")



# Function to authenticate a user
def authenticate(username, password):
    query = "SELECT * FROM users WHERE username = %s AND password = %s"
    cursor.execute(query, (username, hashlib.md5(password.encode()).hexdigest()))
    user = cursor.fetchone()
    return user

#Function to aunthenticate a user
def register(username, password):
    query = "INSERT INTO users (username, password) VALUES (%s, %s)"
    cursor.execute(query, (username, hashlib.md5(password.encode()).hexdigest()))
    cnct.commit()


# Function to handle sign-in
def sign_in():
    username = input("Enter username: ")
    password = getpass.getpass("Enter password: ")
    user = authenticate(username, password)
    if user:
        print("Authentication successful. Welcome, " + username + "!")
        return True
    else:
        print("Authentication failed. Please check your username and password.")
        return False

# Function to handle sign-up
def sign_up():
    username = input("Enter a new username: ")
    password = getpass.getpass("Enter a new password: ")
    register(username, password)
    print("Registration successful. You can now sign in.")


# Function to handle doctor-related options
def doctor_menu():
    while True:
        print("Doctor Management")
        print("1. Add a doctor")
        print("2. Update doctor details")
        print("3. Delete a doctor")
        print("4. Print Doctors")
        print("5. Back to main menu")
        choice = input("Enter your choice: ")

        if choice == '1':
            add_doctor()
        elif choice == '2':
            update_doctor()
        elif choice == '3':
            delete_doctor()
        elif choice == '4':
            print_table("doctors")
        elif choice == '5':
            break
        else:
            print("Invalid choice. Please try again.")

# Function to handle nurse-related options
def nurse_menu():
    while True:
        print("Nurse Management")
        print("1. Add a nurse")
        print("2. Update nurse details")
        print("3. Delete a nurse")
        print("4. Print Nurses")
        print("5. Back to main menu")
        choice = input("Enter your choice: ")

        if choice == '1':
            add_nurse()
        elif choice == '2':
            update_nurse()
        elif choice == '3':
            delete_nurse()
        elif choice == '4':
            print_table("nurses")
        elif choice == '5':
            break
        else:
            print("Invalid choice. Please try again.")

# Function to handle patient-related options
def patient_menu():
    while True:
        print("Patient Management")
        print("1. Add a patient")
        print("2. Update patient details")
        print("3. Delete a patient")
        print("4. Print Patients")
        print("5. Back to main menu")
        choice = input("Enter your choice: ")

        if choice == '1':
            add_patient()
        elif choice == '2':
            update_patient()
        elif choice == '3':
            delete_patient()
        elif choice == '4':
            print_table("patients")
        elif choice == '5':
            break
        else:
            print("Invalid choice. Please try again.")

# Function to handle staff-related options
def staff_menu():
    while True:
        print("Staff Management")
        print("1. Add a staff member")
        print("2. Update staff details")
        print("3. Delete a staff member")
        print("4. Print Staff Members")
        print("5. Back to main menu")
        choice = input("Enter your choice: ")

        if choice == '1':
            add_staff()
        elif choice == '2':
            update_staff()
        elif choice == '3':
            delete_staff()
        elif choice == '4':
            print_table("staff")
        elif choice == '5':
            break
        else:
            print("Invalid choice. Please try again.")

# Function to handle appointment-related options
def appointment_menu():
    while True:
        print("Appointment Management")
        print("1. Book an appointment")
        print("2. Update appointment details")
        print("3. Delete an appointment")
        print("4. Print Appointments")
        print("5. Back to main menu")
        choice = input("Enter your choice: ")

        if choice == '1':
            book_appointment()
        elif choice == '2':
            update_appointment()
        elif choice == '3':
            delete_appointment()
        elif choice == '4':
            print_table("appointments")
        elif choice == '5':
            break
        else:
            print("Invalid choice. Please try again.")

# Function to handle billing-related options
def billing_menu():
    while True:
        print("Billing Management")
        print("1. Add billing information")
        print("2. Update billing details")
        print("3. Delete billing information")
        print("4. Print Billing Information")
        print("5. Back to main menu")
        choice = input("Enter your choice: ")

        if choice == '1':
            add_billing()
        elif choice == '2':
            update_billing()
        elif choice == '3':
            delete_billing()
        elif choice == '4':
            print_table_contents("billing")
        elif choice == '5':
            break
        else:
            print("Invalid choice. Please try again.")


# Function to handle main menu
def main_menu():
    while True:
        print("******************************Akarya HealthHub -""'मरते दम तक आपके साथ बाकी सब भगवान के हाथ'*********************************")
        print("1. Sign in")
        print("2. Sign up")
        print("3. Exit")
        choice = input("Enter your choice: ")

        if choice == '1':
            if sign_in():
                while True:
                    print("******************************Akarya HealthHub -""'मरते दम तक आपके साथ बाकी सब भगवान के हाथ'*********************************")
                    print("1. Doctor Management")
                    print("2. Patient Management")
                    print("3. Nurse Management")
                    print("4. Staff Management")
                    print("5. Appointment Management")
                    print("6. Billing Management")
                    print("7. Sign out")
                    choice = input("Enter your choice: ")

                    if choice == '1':
                        doctor_menu()
                    elif choice == '2':
                        patient_menu()
                    elif choice == '3':
                        nurse_menu()
                    elif choice == '4':
                        staff_menu()
                    elif choice == '5':
                        appointment_menu()
                    elif choice == '6':
                        billing_menu()
                    elif choice == '7':
                        print("Signing out.")
                        break
                    else:
                        print("Invalid choice. Please try again.")

        elif choice == '2':
            sign_up()  # Call the sign_up function here
        elif choice == '3':
            break
        else:
            print("Invalid choice. Please try again.")

# Main program :
if __name__ == "__main__":
    main_menu()

# Close the cursor and connection            
cursor.close()
cnct.close()